In [1]:
# Imports
import pickle
import pandas as pd
import networkx as nx
import community as community_louvain
import os

os.makedirs("output", exist_ok=True)

In [2]:
# --- Load the saved graph ---
with open("road_graph.pkl", "rb") as f:
    G_main = pickle.load(f)

# --- Largest weakly connected component ---
largest_cc = max(nx.weakly_connected_components(G_main), key=len)
G_largest = G_main.subgraph(largest_cc).copy()

print(f"Largest WCC: {len(G_largest.nodes())} nodes, {len(G_largest.edges())} edges")

Largest WCC: 162634 nodes, 399630 edges


In [3]:
# --- HITS (Hubs & Authorities) ---
hubs, authorities = nx.hits(G_largest, max_iter=500)

pd.DataFrame(hubs.items(), columns=["node", "hub_score"]).to_csv("output/hubs.csv", index=False)
pd.DataFrame(authorities.items(), columns=["node", "authority_score"]).to_csv("output/authorities.csv", index=False)

top_hubs = sorted(hubs.items(), key=lambda x: x[1], reverse=True)[:10]
top_auth = sorted(authorities.items(), key=lambda x: x[1], reverse=True)[:10]

print("Top 10 Hubs:")
for n, s in top_hubs: print(n, round(s, 5))
print("\nTop 10 Authorities:")
for n, s in top_auth: print(n, round(s, 5))

Top 10 Hubs:
4466994312 0.08456
3435239188 0.08388
3435245119 0.08388
3435245097 0.08249
4466994311 0.08249
3435239187 0.07912
4466994306 0.07912
3435245112 0.07705
3435245121 0.07705
3435245106 0.07368

Top 10 Authorities:
4466994312 0.08456
3435245119 0.08388
3435239188 0.08388
3435245097 0.08249
4466994311 0.08249
3435239187 0.07912
4466994306 0.07912
3435245112 0.07705
3435245121 0.07705
3435245106 0.07368


In [4]:
# --- Louvain Community Detection ---
partition = community_louvain.best_partition(G_largest.to_undirected())
pd.DataFrame(partition.items(), columns=["node", "community"]).to_csv("output/communities.csv", index=False)

print("\n✅ HITS + Communities exported to /output/")


✅ HITS + Communities exported to /output/
